In [2]:
# !pip3 install optuna

In [3]:
import optuna
import subprocess
import os
import sys
import math
import logging

/nfs/TUEIEDAscratch/ge85zic/venv_layer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import optuna
import subprocess
import os
import sys
import math
import logging

# --- Configuration ---
GEN_MODEL_SCRIPT = "gen_model.py"
NUM_TRIALS = 10
OUTPUT_DIR = "generated_models"
# ✅ DEFINE A STATIC MAXIMUM NUMBER OF LAYERS
MAX_LAYERS = 6 
SHAPE_TYPE = "3d"
# --- Setup Logging ---
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- Helper Function (Unchanged) ---
def calculate_conv_output_dim(input_dim, kernel, padding, stride):
    if padding.upper() == 'SAME':
        return math.ceil(input_dim / stride)
    elif padding.upper() == 'VALID':
        return math.ceil((input_dim - kernel + 1) / stride)
    else:
        raise ValueError(f"Unsupported padding type: {padding}")

def objective(trial: optuna.trial.Trial):
    """
    Optuna objective function with a static search space.
    """
    model_filename = os.path.join(OUTPUT_DIR, f"model_trial_{trial.number}.tflite")

    # ===================================================================
    # 1. DEFINE THE COMPLETE, STATIC SEARCH SPACE
    # ===================================================================
    # This section defines all possible hyperparameters that *could* be used,
    # up to MAX_LAYERS. This ensures the search space is identical for every trial.
    
    # --- Input Shape Definition ---
    shape_type = SHAPE_TYPE #trial.suggest_categorical("shape_type", ["3d", "1d"])
    if shape_type == "3d":
        h = trial.suggest_categorical("in_height", [16, 32, 64, 96, 128,256,512,1024])
        w = trial.suggest_categorical("in_width", [16, 32, 64, 96, 128,256,512,1024])
        c = trial.suggest_categorical("in_channels", [1, 3, 16])
        initial_shape = (h, w, c)
        in_shape_str = f"{h},{w},{c}"
    else: # 1d
        dim = trial.suggest_categorical("in_dim", [256, 512, 1024, 2048])
        initial_shape = (dim,)
        in_shape_str = str(dim)

    # --- Number of layers to *actually use* in this trial ---
    num_layers = MAX_LAYERS
    
    # --- Define hyperparameters for ALL potential layers ---
    all_3d_ops = ["conv2d", "depthwise_conv2d", "avg_pool2d", "max_pool2d", "flatten"]
    for i in range(MAX_LAYERS):
        # Layer type
        trial.suggest_categorical(f"l{i}_op", all_3d_ops + ["fully_connected"])
        
        # Conv2D params
        trial.suggest_int(f"l{i}_conv_filters", 8, 64)
        trial.suggest_categorical(f"l{i}_conv_kernel", [1, 3, 5])
        trial.suggest_categorical(f"l{i}_conv_stride", [1, 2])
        
        # Depthwise Conv2D params
        trial.suggest_categorical(f"l{i}_dw_multiplier", [1, 2, 4])
        trial.suggest_categorical(f"l{i}_dw_kernel", [3, 5])
        trial.suggest_categorical(f"l{i}_dw_stride", [1, 2])

        # General Conv/Pool params
        trial.suggest_categorical(f"l{i}_padding", ["SAME", "VALID"])
        trial.suggest_categorical(f"l{i}_activation", ["relu", ""])
        trial.suggest_categorical(f"l{i}_bias", [0, 1])

        # Pooling params
        trial.suggest_categorical(f"l{i}_pool_size", [2, 3])
        trial.suggest_categorical(f"l{i}_pool_stride", [1, 2])
        
        # Fully Connected params
        trial.suggest_int(f"l{i}_fc_units", 10, 512)

    # ===================================================================
    # 2. CONSTRUCT THE MODEL USING THE CHOSEN HYPERPARAMETERS
    # ===================================================================
    # This section now *uses* the values defined above to build the command.
    # The loop only runs up to `num_layers`.
    
    cmd_args = ["python3", GEN_MODEL_SCRIPT, "tflite", model_filename, in_shape_str, str(num_layers)]
    current_shape = initial_shape
    is_1d_model = (len(current_shape) == 1)

    for i in range(num_layers):
        # Retrieve the chosen layer operation for the current layer
        layer_op = trial.params[f"l{i}_op"]

        # --- Validate the chosen layer op ---
        if is_1d_model and layer_op != "fully_connected":
            raise optuna.exceptions.TrialPruned(f"Layer {i} must be fully_connected after flatten/1D input.")
        if not is_1d_model and layer_op == "fully_connected":
             raise optuna.exceptions.TrialPruned(f"Cannot use fully_connected on 3D data without a flatten layer.")
        if i == num_layers - 1 and layer_op == "flatten":
            raise optuna.exceptions.TrialPruned("Cannot use 'flatten' as the final layer.")

        # --- Build attributes for the chosen layer op ---
        if layer_op == "conv2d":
            filters = trial.params[f"l{i}_conv_filters"]
            kernel = trial.params[f"l{i}_conv_kernel"]
            stride = trial.params[f"l{i}_conv_stride"]
            padding = trial.params[f"l{i}_padding"]
            activation = trial.params[f"l{i}_activation"]
            use_bias = trial.params[f"l{i}_bias"]
            attrs = f"{filters},{kernel},{kernel},{stride},{stride},1,1,{padding},channels_last,{activation},{use_bias},1"
            cmd_args.extend([layer_op, attrs])
            h, w, _ = current_shape
            current_shape = (calculate_conv_output_dim(h, kernel, padding, stride),
                             calculate_conv_output_dim(w, kernel, padding, stride),
                             filters)
        elif layer_op == "depthwise_conv2d":
            multiplier = trial.params[f"l{i}_dw_multiplier"]
            kernel = trial.params[f"l{i}_dw_kernel"]
            stride = trial.params[f"l{i}_dw_stride"]
            padding = trial.params[f"l{i}_padding"]
            activation = trial.params[f"l{i}_activation"]
            use_bias = trial.params[f"l{i}_bias"]
            attrs = f"{multiplier},{kernel},{kernel},{stride},{stride},1,1,{padding},channels_last,{activation},{use_bias}"
            cmd_args.extend([layer_op, attrs])
            h, w, c = current_shape
            current_shape = (calculate_conv_output_dim(h, kernel, padding, stride),
                             calculate_conv_output_dim(w, kernel, padding, stride),
                             c * multiplier)
        elif layer_op in ["avg_pool2d", "max_pool2d"]:
            pool_size = trial.params[f"l{i}_pool_size"]
            stride = trial.params[f"l{i}_pool_stride"]
            padding = trial.params[f"l{i}_padding"]
            attrs = f"{pool_size},{pool_size},{stride},{stride},{padding},channels_last"
            cmd_args.extend([layer_op, attrs])
            h, w, c = current_shape
            current_shape = (calculate_conv_output_dim(h, pool_size, padding, stride),
                             calculate_conv_output_dim(w, pool_size, padding, stride),
                             c)
        elif layer_op == "flatten":
            cmd_args.extend(["flatten", "_"])
            h, w, c = current_shape
            current_shape = (h * w * c,)
            is_1d_model = True
        elif layer_op == "fully_connected":
            units = trial.params[f"l{i}_fc_units"]
            activation = trial.params[f"l{i}_activation"] # Re-using conv activation
            use_bias = trial.params[f"l{i}_bias"]       # Re-using conv bias
            attrs = f",{units},{activation},{use_bias}"
            cmd_args.extend([layer_op, attrs])
            current_shape = (units,)

    # ===================================================================
    # 3. Execute the command (Unchanged)
    # ===================================================================
    print("\n" + "-"*40)
    print(f"Trial {trial.number}: Executing command for a model with {num_layers} layers...")
    print(" ".join(map(str, cmd_args)))
    print("-"*40)
    try:
        subprocess.run(cmd_args, check=True, capture_output=True, text=True)
        print(f"✅ Trial {trial.number}: Model generated successfully.")
        return 0.0
    except subprocess.CalledProcessError as e:
        print(f"❌ Trial {trial.number}: Model generation FAILED.")
        print(f"   Stderr: {e.stderr.strip()}")
        raise optuna.exceptions.TrialPruned()

# --- (The main() function does not need to be changed) ---
def main():
    if not os.path.exists(GEN_MODEL_SCRIPT):
        print(f"Error: The script '{GEN_MODEL_SCRIPT}' was not found.")
        sys.exit(1)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    study = optuna.create_study(
        direction="minimize",
        study_name="tflite-model-generator_new",
        storage="sqlite:///tflite_generator.db",
        load_if_exists=True,
        pruner=optuna.pruners.MedianPruner()
    )
    try:
        study.optimize(objective, n_trials=NUM_TRIALS, timeout=600)
    except KeyboardInterrupt:
        print("Study interrupted by user.")
    
    print("\n" + "="*50)
    print("Study statistics: ")
    print(f"  Number of finished trials: {len(study.trials)}")
    pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
    print(f"  Pruned (failed/invalid) trials: {len(pruned_trials)}")
    print(f"  Complete (successful) trials: {len(complete_trials)}")
    if complete_trials:
        print("\nAn example of a successful trial's parameters:")
        # Filter params to only show those used in the trial
        best_trial = complete_trials[0]
        used_params = {k: v for k, v in best_trial.params.items() if not k.startswith(f"l{best_trial.params['num_layers']}"
)}
        print(used_params)
    print("="*50)

if __name__ == "__main__":
    main()


----------------------------------------
Trial 8: Executing command for a model with 6 layers...
python3 gen_model.py tflite generated_models/model_trial_8.tflite 96,16,16 6 conv2d 15,3,3,2,2,1,1,VALID,channels_last,,0,1 depthwise_conv2d 4,3,3,2,2,1,1,VALID,channels_last,relu,0 max_pool2d 3,3,1,1,VALID,channels_last conv2d 45,1,1,2,2,1,1,VALID,channels_last,,0,1 depthwise_conv2d 2,5,5,1,1,1,1,VALID,channels_last,,0 depthwise_conv2d 1,5,5,1,1,1,1,VALID,channels_last,relu,0
----------------------------------------
❌ Trial 8: Model generation FAILED.
   Stderr: 2025-10-16 21:32:03.434094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-16 21:32:06.842862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorR

[W 2025-10-16 21:35:05,224] Trial 69 failed with parameters: {'in_height': 96, 'in_width': 96, 'in_channels': 16, 'l0_op': 'conv2d', 'l0_conv_filters': 60, 'l0_conv_kernel': 3, 'l0_conv_stride': 1, 'l0_dw_multiplier': 1, 'l0_dw_kernel': 5, 'l0_dw_stride': 2, 'l0_padding': 'SAME', 'l0_activation': '', 'l0_bias': 0, 'l0_pool_size': 2, 'l0_pool_stride': 2, 'l0_fc_units': 281, 'l1_op': 'conv2d', 'l1_conv_filters': 18, 'l1_conv_kernel': 5, 'l1_conv_stride': 1, 'l1_dw_multiplier': 4, 'l1_dw_kernel': 5, 'l1_dw_stride': 2, 'l1_padding': 'VALID', 'l1_activation': 'relu', 'l1_bias': 0, 'l1_pool_size': 2, 'l1_pool_stride': 1, 'l1_fc_units': 459, 'l2_op': 'conv2d', 'l2_conv_filters': 19, 'l2_conv_kernel': 1, 'l2_conv_stride': 1, 'l2_dw_multiplier': 1, 'l2_dw_kernel': 3, 'l2_dw_stride': 2, 'l2_padding': 'VALID', 'l2_activation': 'relu', 'l2_bias': 1, 'l2_pool_size': 2, 'l2_pool_stride': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/nfs/TUEIEDAsc

Trial 69 failed with parameters: {'in_height': 96, 'in_width': 96, 'in_channels': 16, 'l0_op': 'conv2d', 'l0_conv_filters': 60, 'l0_conv_kernel': 3, 'l0_conv_stride': 1, 'l0_dw_multiplier': 1, 'l0_dw_kernel': 5, 'l0_dw_stride': 2, 'l0_padding': 'SAME', 'l0_activation': '', 'l0_bias': 0, 'l0_pool_size': 2, 'l0_pool_stride': 2, 'l0_fc_units': 281, 'l1_op': 'conv2d', 'l1_conv_filters': 18, 'l1_conv_kernel': 5, 'l1_conv_stride': 1, 'l1_dw_multiplier': 4, 'l1_dw_kernel': 5, 'l1_dw_stride': 2, 'l1_padding': 'VALID', 'l1_activation': 'relu', 'l1_bias': 0, 'l1_pool_size': 2, 'l1_pool_stride': 1, 'l1_fc_units': 459, 'l2_op': 'conv2d', 'l2_conv_filters': 19, 'l2_conv_kernel': 1, 'l2_conv_stride': 1, 'l2_dw_multiplier': 1, 'l2_dw_kernel': 3, 'l2_dw_stride': 2, 'l2_padding': 'VALID', 'l2_activation': 'relu', 'l2_bias': 1, 'l2_pool_size': 2, 'l2_pool_stride': 1} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/nfs/TUEIEDAscratch/ge85zic/venv_layer/lib

[W 2025-10-16 21:35:05,294] Trial 69 failed with value None.


Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Trial 69 failed with value None.
Study interrupted by user.

Study statistics: 
  Number of finished trials: 70
  Pruned (failed/invalid) trials: 46
  Complete (successful) trials: 23

An example of a successful trial's parameters:


KeyError: 'num_layers'